##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 生成随机数

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/random_numbers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在  TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/random_numbers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/random_numbers.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/random_numbers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a></td>
</table>

TensorFlow 在 `tf.random` 模块中提供了一组伪随机数生成器 (RNG)。本文介绍如何控制随机数生成器，以及这些生成器如何与其他 Tensorflow 子系统交互。

Note: The random numbers are not guaranteed to be consistent across TensorFlow versions. See: [Version Compatibility](https://www.tensorflow.org/guide/versions#what_is_not_covered)

TensorFlow 提供了两种方法来控制随机数生成过程：

1. 通过明确使用 `tf.random.Generator` 对象。每个此类对象都会在 `tf.Variable` 中维护一个状态，该状态在每次生成随机数后都会发生改变。

2. 通过使用纯函数式无状态随机函数，如 `tf.random.stateless_uniform`。在同一设备上调用具有相同参数（包括种子）的这些函数会产生相同的结果。

警告：目前尚未弃用 TF 1.x 中的旧版 RNG（如 `tf.random.uniform` 和 `tf.random.normal`），但强烈建议不要使用。

## 设置

In [ ]:
import tensorflow as tf

# Creates 2 virtual devices cpu:0 and cpu:1 for using distribution strategy
physical_devices = tf.config.experimental.list_physical_devices("CPU")
tf.config.experimental.set_virtual_device_configuration(
    physical_devices[0], [
        tf.config.experimental.VirtualDeviceConfiguration(),
        tf.config.experimental.VirtualDeviceConfiguration()
    ])

## `tf.random.Generator` 类

当您希望每次调用 RNG 都产生不同的结果时，可以使用 `tf.random.Generator` 类。它会维护一个内部状态（由 `tf.Variable` 对象管理），该状态在每次生成随机数时都会更新。由于该状态由 `tf.Variable` 管理，因此，它可以利用 `tf.Variable` 提供的所有功能，如简单的检查点、自动控制依赖项和线程安全性。

通过手动创建 `tf.random.Generator`类的一个对象，您可以获得该生成器，或者通过调用 `tf.random.get_global_generator()`，您可以获得默认全局生成器：

In [ ]:
g1 = tf.random.Generator.from_seed(1)
print(g1.normal(shape=[2, 3]))
g2 = tf.random.get_global_generator()
print(g2.normal(shape=[2, 3]))

有多种方法可以创建生成器对象。最简单的方法是使用 `Generator.from_seed`（代码如上），从种子创建生成器。种子可以是任何非负整数，`from_seed` 还有一个可选参数 `alg`，这是该生成器将使用的 RNG 算法。

In [ ]:
g1 = tf.random.Generator.from_seed(1, alg='philox')
print(g1.normal(shape=[2, 3]))

有关详细信息，请参阅后文中的*算法*部分。

创建生成器的另一种方法是使用 `Generator.from_non_deterministic_state`。以这种方式创建的生成器首先会处于非确定状态，具体取决于时间和操作系统等因素。

In [ ]:
g = tf.random.Generator.from_non_deterministic_state()
print(g.normal(shape=[2, 3]))

还有其他方法可以创建生成器，比如说通过显式状态创建，本指南不作赘述。

当使用 `tf.random.get_global_generator` 来获取全局生成器时，需要注意设备放置。第一次调用 `tf.random.get_global_generator` 时就会创建全局生成器（从非确定状态），并将其放置在该调用的作用域内的默认设备上。举个例子，如果第一次调用 `tf.random.get_global_generator` 的位置在 `tf.device("gpu")` 作用域内，则会将全局生成器放置在 GPU 上，如果稍后要从 CPU 使用全局生成器，则会将其从 GPU 复制到 CPU。

There is also a function `tf.random.set_global_generator` for replacing the global generator with another generator object. This function should be used with caution though, because the old global generator may have been captured by a `tf.function` (as a weak reference), and replacing it will cause it to be garbage collected, breaking the `tf.function`. A better way to reset the global generator is to use one of the "reset" functions such as `Generator.reset_from_seed`, which won't create new generator objects.

In [ ]:
g = tf.random.Generator.from_seed(1)
print(g.normal([]))
print(g.normal([]))
g.reset_from_seed(1)
print(g.normal([]))

### 创建独立的随机数流

许多应用都需要多个独立的随机数流，所谓独立，就是指不能相互重叠，也不能有统计学上可检测到的相关性。通过使用 `Generator.split` 创建多个一定相互独立的生成器即可实现此目的（即生成独立流）。

In [ ]:
g = tf.random.Generator.from_seed(1)
print(g.normal([]))
new_gs = g.split(3)
for new_g in new_gs:
  print(new_g.normal([]))
print(g.normal([]))

与 `normal` 之类的 RNG 方法类似，`split` 会改变调用它的生成器的状态（上例中为 `g`）。除相互之间保持独立外，新生成器 (`new_gs`) 还一定独立于旧生成器 (`g`)。

当您想要确保使用的生成器位于与其他计算相同的设备上，从而避免跨设备复制的开销时，生成新生成器也很有用。例如： 

In [ ]:
with tf.device("cpu"):  # change "cpu" to the device you want
  g = tf.random.get_global_generator().split(1)[0]  
  print(g.normal([]))  # use of g won't cause cross-device copy, unlike the global generator

注：在理论上，此处可以使用 `from_seed`（而不是 `split`）之类的构造函数获取新生成器，但这样做无法保证新生成器与全局生成器相互独立。同时也有使用同一种子或导致产生重叠随机数流的种子意外创建两个生成器的风险。

您可以在拆分生成器上调用 `split`，执行递归拆分。递归深度没有限制（除非发生整数溢出）。

### 与 `tf.function` 交互

与 `tf.function` 一起使用时，`tf.random.Generator` 遵循与 `tf.Variable` 相同的原则。这包括三个方面：

#### 在 `tf.function` 的外部创建生成器

`tf.function` 可以使用在其外部创建的生成器。

In [ ]:
g = tf.random.Generator.from_seed(1)
@tf.function
def foo():
  return g.normal([])
print(foo())

调用该函数时，用户需要确保生成器对象仍处于活动状态（没有被回收）。

#### 在 `tf.function` 的内部创建生成器

只有 `tf.function` 第一次运行时，才可以在其内部创建生成器。 

In [ ]:
g = None
@tf.function
def foo():
  global g
  if g is None:
    g = tf.random.Generator.from_seed(1)
  return g.normal([])
print(foo())
print(foo())

#### 将生成器作为参数传递给 `tf.function`

When used as an argument to a `tf.function`, different generator objects will cause retracing of the `tf.function`.

In [ ]:
num_traces = 0
@tf.function
def foo(g):
  global num_traces
  num_traces += 1
  return g.normal([])
foo(tf.random.Generator.from_seed(1))
foo(tf.random.Generator.from_seed(2))
print(num_traces)

Note that this retracing behavior is consistent with `tf.Variable`:

In [ ]:
num_traces = 0
@tf.function
def foo(v):
  global num_traces
  num_traces += 1
  return v.read_value()
foo(tf.Variable(1))
foo(tf.Variable(2))
print(num_traces)

### 与分布策略交互

There are two ways in which `Generator` interacts with distribution strategies.

#### 在分布策略的外部创建生成器

如果是在策略作用域的外部创建的生成器，则会序列化访问此生成器的所有副本，因此，每一个副本都会得到不同的随机数。

In [ ]:
g = tf.random.Generator.from_seed(1)
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat.scope():
  def f():
    print(g.normal([]))
  results = strat.run(f)

请注意，这种使用方法可能产生性能问题，因为生成器的设备与副本不同。

#### 在分布策略的内部创建生成器

If a generator is created inside a strategy scope, each replica will get a different and independent stream of random numbers.

In [ ]:
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat.scope():
  g = tf.random.Generator.from_seed(1)
  print(strat.run(lambda: g.normal([])))
  print(strat.run(lambda: g.normal([])))

Note: Currently `tf.random.Generator` doesn't provide an option to let different replicas get identical (instead of different) streams (which is technically not hard). If you have a use case for this feature, please let the TensorFlow developers know.

If the generator is seeded (e.g. created by `Generator.from_seed`), the random numbers are determined by the seed, even though different replicas get different and uncorrelated numbers. One can think of a random number generated on a replica as a hash of the replica ID and a "primary" random number that is common to all replicas. Hence, the whole system is still deterministic.

`tf.random.Generator` can also be created inside `Strategy.run`:

In [ ]:
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat.scope():
  def f():
    g = tf.random.Generator.from_seed(1)
    a = g.normal([])
    b = g.normal([])
    return tf.stack([a, b])
  print(strat.run(f))
  print(strat.run(f))

We no longer recommend passing `tf.random.Generator` as arguments to `Strategy.run`, because `Strategy.run` generally expects the arguments to be tensors, not generators.

### Saving generators

Generally for saving or serializing you can handle a `tf.random.Generator` the same way you would handle a `tf.Variable` or a `tf.Module` (or its subclasses). In TF there are two mechanisms for serialization: [Checkpoint](https://www.tensorflow.org/guide/checkpoint) and [SavedModel](https://www.tensorflow.org/guide/saved_model).

#### Checkpoint

Generators can be freely saved and restored using `tf.train.Checkpoint`. The random-number stream from the restoring point will be the same as that from the saving point. 

In [ ]:
filename = "./checkpoint"
g = tf.random.Generator.from_seed(1)
cp = tf.train.Checkpoint(generator=g)
print(g.normal([]))

In [ ]:
cp.write(filename)
print("RNG stream from saving point:")
print(g.normal([]))
print(g.normal([]))

In [ ]:
cp.restore(filename)
print("RNG stream from restoring point:")
print(g.normal([]))
print(g.normal([]))

You can also save and restore within a distribution strategy:

In [ ]:
filename = "./checkpoint"
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat.scope():
  g = tf.random.Generator.from_seed(1)
  cp = tf.train.Checkpoint(my_generator=g)
  print(strat.run(lambda: g.normal([])))

In [ ]:
with strat.scope():
  cp.write(filename)
  print("RNG stream from saving point:")
  print(strat.run(lambda: g.normal([])))
  print(strat.run(lambda: g.normal([])))

In [ ]:
with strat.scope():
  cp.restore(filename)
  print("RNG stream from restoring point:")
  print(strat.run(lambda: g.normal([])))
  print(strat.run(lambda: g.normal([])))

You should make sure that the replicas don't diverge in their RNG call history (e.g. one replica makes one RNG call while another makes two RNG calls) before saving. Otherwise, their internal RNG states will diverge and `tf.train.Checkpoint` (which only saves the first replica's state) won't properly restore all the replicas.

You can also restore a saved checkpoint to a different distribution strategy with a different number of replicas. Because a `tf.random.Generator` object created in a strategy can only be used in the same strategy, to restore to a different strategy, you have to create a new `tf.random.Generator` in the target strategy and a new `tf.train.Checkpoint` for it, as shown in this example:

In [ ]:
filename = "./checkpoint"
strat1 = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat1.scope():
  g1 = tf.random.Generator.from_seed(1)
  cp1 = tf.train.Checkpoint(my_generator=g1)
  print(strat1.run(lambda: g1.normal([])))

In [ ]:
with strat1.scope():
  cp1.write(filename)
  print("RNG stream from saving point:")
  print(strat1.run(lambda: g1.normal([])))
  print(strat1.run(lambda: g1.normal([])))

In [ ]:
strat2 = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1", "cpu:2"])
with strat2.scope():
  g2 = tf.random.Generator.from_seed(1)
  cp2 = tf.train.Checkpoint(my_generator=g2)
  cp2.restore(filename)
  print("RNG stream from restoring point:")
  print(strat2.run(lambda: g2.normal([])))
  print(strat2.run(lambda: g2.normal([])))

Although `g1` and `cp1` are different objects from `g2` and `cp2`, they are linked via the common checkpoint file `filename` and object name `my_generator`. Overlapping replicas between strategies (e.g. `cpu:0` and `cpu:1` above) will have their RNG streams properly restored like in previous examples. This guarantee doesn't cover the case when a generator is saved in a strategy scope and restored outside of any strategy scope or vice versa, because a device outside strategies is treated as different from any replica in a strategy.

#### SavedModel

`tf.random.Generator` can be saved to a SavedModel. The generator can be created within a strategy scope. The saving can also happen within a strategy scope. 

In [ ]:
filename = "./saved_model"

class MyModule(tf.Module):

  def __init__(self):
    super(MyModule, self).__init__()
    self.g = tf.random.Generator.from_seed(0)

  @tf.function
  def __call__(self):
    return self.g.normal([])

  @tf.function
  def state(self):
    return self.g.state

strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat.scope():
  m = MyModule()
  print(strat.run(m))
  print("state:", m.state())

In [ ]:
with strat.scope():
  tf.saved_model.save(m, filename)
  print("RNG stream from saving point:")
  print(strat.run(m))
  print("state:", m.state())
  print(strat.run(m))
  print("state:", m.state())

In [ ]:
imported = tf.saved_model.load(filename)
print("RNG stream from loading point:")
print("state:", imported.state())
print(imported())
print("state:", imported.state())
print(imported())
print("state:", imported.state())

Loading a SavedModel containing `tf.random.Generator` into a distribution strategy is not recommended because the replicas will all generate the same random-number stream (which is because replica ID is frozen in SavedModel's graph). 

Loading a distributed `tf.random.Generator` (a generator created within a distribution strategy) into a non-strategy environment, like the above example, also has a caveat. The RNG state will be properly restored, but the random numbers generated will be different from the original generator in its strategy (again because a device outside strategies is treated as different from any replica in a strategy).

## 无状态 RNG

无状态 RNG 的使用方法非常简单。因为它们是纯函数，不涉及状态或副作用。

In [ ]:
print(tf.random.stateless_normal(shape=[2, 3], seed=[1, 2]))
print(tf.random.stateless_normal(shape=[2, 3], seed=[1, 2]))

每个无状态 RNG 都需要一个 `seed` 参数，该参数必须是形状为 `[2]` 的整数张量。该运算的结果完全由种子确定。

The RNG algorithm used by stateless RNGs is device-dependent, meaning the same op running on a different device may produce different outputs.

## 算法

### 基本信息

`tf.random.Generator` 类和 `stateless` 函数在所有设备上都支持 Philox 算法（写作 `"philox"` 或 `tf.random.Algorithm.PHILOX`）。

如果使用相同的算法且从相同的状态开始，则不同的设备会生成相同的整数。它们还可以生成“几乎相同”的浮点数，虽然由于设备执行浮点计算的方式不同（如降阶），数值可能存在微小的差异。

### XLA 设备

在 XLA 驱动的设备（如 TPU 以及启用 XLA 时的 CPU/GPU）上，还支持 ThreeFry 算法（写作 `"threefry"` 或 `tf.random.Algorithm.THREEFRY`）。与 Philox 算法相比，该算法在 TPU 上执行速度较快，而在 CPU/GPU 上执行速度较慢。 

有关这些算法的更多详细信息，请参阅论文[“Parallel Random Numbers: As Easy as 1, 2, 3”](https://www.thesalmons.org/john/random123/papers/random123sc11.pdf)。